In [113]:
import bs4 as bs
import urllib.request
import csv
import pandas as pd
import re

my_url = urllib.request.urlopen('https://store.steampowered.com/search/?os=win%2Cmac&supportedlang=english&specials=1&ndl=1').read()


soup = bs.BeautifulSoup(my_url, 'html.parser')
def remove_tags(text):
    return ''.join(xml.etree.ElementTree.fromstring(text).itertext())


# All soup findings, might be an irrelevant in there
data_search_results = (soup.find_all('a', {'class':'search_result_row ds_collapse_flag'}))

data_discounts_percent = (soup.find_all('div', {'class':'col search_discount responsive_secondrow'}))
data_discounts_euro = (soup.find_all('div', {'class':'col search_price discounted responsive_secondrow'}))
data_release_date = (soup.find_all('div', {'class':'col search_released responsive_secondrow'}))

data_rating_div = (soup.find_all('span', {'class':'search_review_summary'}))
data_OSSup = (soup.find_all('div', {'class':'col search_name ellipsis'}))

data_rating = (soup.find_all('td', {'data-tooltip': True}))
data_title = (soup.find_all('span', {'class':'title'}))




In [144]:
# All empty lists to use
data_title_list = []
data_rating_list = []
data_reviews_list = []
data_discounts_percent_list = []
data_discountPrice_list = []
data_normalPrice_list = []
data_release_date_list = []
data_win_list = []
data_lin_list = []
data_mac_list = []

#pattern is irrelevant, just an attempt to filter out strings
pattern = r'[^A-Za-z0-9]+'
str2 = " \n"

# All my functions for creating different lists, used the "normal" list creator where I could
def filter_string(str1, str2):
    for c in str2:
        str1 = str1.replace(c, '')
    return str1

def filter_strike(str1, strike):
    for c in strike:
        str1 = str1.replace(c, '')

def create_list(data, data_list):
    
    for item in data:

        string = filter_string(item.text, str2)

        data_list.append(string)
    return data_list

def create_DiscountPriceList(data, data_list):
    for item in data:
        

        string = filter_string(item.text, str2)
        data_list.append(string.split("€",1)[1])

    return data_list

def create_NormalPriceList(data, data_list):
    for item in data:

        strike = item.find('strike')
        string = filter_string(strike.text, str2)
        data_list.append(string)
    return data_list

def create_RatingList(data, data_list):
    for item in data:
        string = item['data-tooltip-html']
        data_list.append(string.split("<",1)[0])
    return data_list

def create_ReviewsList(data, data_list):
    for item in data:
        string = item['data-tooltip-html']
        numbers = [int(s) for s in string.replace(',','').split() if s.isdigit()]
        data_list.append(numbers[0])
    return data_list

def OS_Support(data):
    for item in data:
        spanWin = item.find('span', {'class': 'platform_img win'})
        spanLin = item.find('span', {'class': 'platform_img linux'})
        spanMac = item.find('span', {'class': 'platform_img mac'})
        
        winNum = 1
        linNum = 1
        macNum = 1
        
        winNum = 1 if spanWin else 0
        
        linNum = 1 if spanLin else 0
        
        macNum = 1 if spanMac else 0

    
    return winNum, linNum, macNum

def create_OSList(data, data_list, x):
    
    for item in data:   
        value = OS_Support(data)
        data_list.append(value[x])
    return data_list


# All the seperate lists creations
titles = create_list(data_title, data_title_list)
discounts = create_list(data_discounts_percent, data_discounts_percent_list)
discountPrice = create_DiscountPriceList(data_discounts_euro, data_discountPrice_list)
normalPrice = create_NormalPriceList(data_discounts_euro, data_normalPrice_list)
releaseDate = create_list(data_release_date, data_release_date_list)
ratingList = create_RatingList(data_rating_div, data_rating_list)
reviewList = create_ReviewsList(data_rating_div, data_reviews_list)

win = create_OSList(data_OSSup, data_win_list, 0)
lin = create_OSList(data_OSSup, data_lin_list, 1)
mac = create_OSList(data_OSSup, data_mac_list, 2)


# Issue checking
print(len(titles),
len(discounts),
len(discountPrice), #42
len(normalPrice), #42
len(releaseDate),
len(ratingList), #49
len(reviewList),
len(data_rating_div)) #49



 # Difference in list lengths, both price lists doesn't include non sale items while the others does
dataFrameNoPrices = pd.DataFrame({
    'Title': titles,
    'Discount': discounts,
    'Release_date': releaseDate,
#     'Rating': ratingList,
#     'Reviews': reviewList,
    'Win': win,
    'Lin': lin,
    'Mac': mac
})

dataFrameRatRev = pd.DataFrame({
    'Rating': ratingList,
    'Reviews': reviewList
})


dataFramePrices = pd.DataFrame({
    'Price': discountPrice,
    'Normal_Price': normalPrice
})

# Merging the two frames and removing every item without discount. 
dataFrameRatRevPrices = pd.concat([dataFrameNoPrices_Removed, dataFrameRatRev], axis=1, join="inner")
dataFrameNoPrices_Removed = dataFrameRatRevPrices[dataFrameRatRevPrices['Discount'] != '']

# Printing lengths here and there trying to figure out where it is messing up
print(len(dataFrameNoPrices_Removed))
print(len(dataFrameRatRev))

#




resultFrame = pd.concat([dataFrameNoPrices_Removed, dataFrameRatRevPrices], axis=1, join="inner")

# Rearrange the dataframe after concat
resultFrameOrdered = resultFrameNoPrices_Removed[['Title', 'Discount', 'Price', 'Normal_Price','Rating', 'Reviews', 'Release_date', 'Win', 'Lin', 'Mac']]


print(len(resultFrame))



resultRatingNumbers = resultFrameOrdered.replace([
   'Overwhelmingly Positive', 'Very Positive', 'Positive', 'Mostly Positive', 'Mixed', 'Mostly Negative', 'Negative', 'Very Negative', 'Overwhelmingly Negative'],
    ['10','9','8','7','6','5','4','3','2'])


# Putting the dataframe into csv file
resultRatingNumbers.to_csv('steamSales.csv', mode='a', index=False, header=False)
resultRatingNumbers




50 50 43 43 50 49 49 49
42
49
42


,Title,Discount,Price,Normal_Price,Rating,Reviews,Release_date,Win,Lin,Mac
0,"Warhammer40,000:Inquisitor-Martyr",-80%,"9,99€","49,99€",7,13732,"5Jun,2018",1,0,1
1,X42022Bundle,-71%,"39,14€","134,92€",7,12990,,1,0,1
2,X4:Foundations,-60%,"19,99€","49,99€",7,12266,"30Nov,2018",1,0,1
3,Sackboy™:ABigAdventure,-33%,"40,19€","59,99€",9,427,"27Oct,2022",1,0,1
4,SCUM,-50%,"14,99€","29,99€",7,65675,"29Aug,2018",1,0,1
5,CalloftheWild:TheAngler™,-34%,"19,79€","29,99€",6,3158,"31Aug,2022",1,0,1
6,NoMan'sSky,-50%,"27,49€","54,99€",7,195963,"12Aug,2016",1,0,1
7,"Warhammer40,000:Inquisitor-Martyr-SororitasClass",-10%,"13,49€","14,99€",7,75,"28Nov,2022",1,0,1
8,Battlefield™2042,-67%,"19,79€","59,99€",5,120674,"19Nov,2021",1,0,1
9,DestroyAllHumans!2-Reprobed,-25%,"29,99€","39,99€",9,1024,"30Aug,2022",1,0,1


Finns liten bugg pga av att de endast kommer in 49 ratings. Rör om lite listan vid ett ställe och efter det kan man inte riktigt lita på alla columner. Hittade inte varför den gör de men tror det har antingen något med non sale items eller dlc items i steam storen. Blev lite strul med samma pågrund av att priserna som hämtas så där skippas helt icke sale priser. Mac, lin och windows support borde enligt min egna logik i alla fall fungera rätt men verkar inte riktigt så. 

# Extra testing etc


In [50]:
for item in data_OSSup:
    spanWin = item.find('span', {'class': 'platform_img win'})
    spanLin = item.find('span', {'class': 'platform_img linux'})
    spanMac = item.find('span', {'class': 'platform_img mac'})
    
    winNum = 0
    linNum = 0
    macNum = 0
    
    if spanWin != None:
        winNum = 1
    if spanLin != None:
        linNum = 1
    if spanMac != None:
        macNum = 1
    

In [136]:
for item in data_rating_div:
    string = item['data-tooltip-html']
    numbers = [int(s) for s in string.replace(',','').split() if s.isdigit()]
    numbers
    print(numbers)
    
print(len(numbers))

[13732]
[12990]
[12266]
[427]
[65675]
[3158]
[195963]
[75]
[120674]
[1024]
[9327]
[27]
[26200]
[6990]
[201]
[39570]
[5283]
[5886]
[62344]
[151759]
[1614]
[18392]
[5527]
[5189]
[6535]
[239]
[64051]
[5698]
[171]
[867]
[8298]
[2068]
[2437]
[7084]
[26365]
[1516]
[3457]
[635]
[10532]
[294]
[114063]
[11661]
[16456]
[19470]
[21129]
[10811]
[418431]
[98]
[22958]
1
